# 消息传递

1.通过消息函数(u,e,v) => mailbox  
2.聚合函数(mailbox)  
3.更新函数(x,mailbox)=>x



# 消息函数

利用消息函数，我们可以将两个节点特征相结合然后赋给边，也可以将节点和边特征相结合然后赋给边，即：node+node->edge或node+edge->edge。

# 聚合函数

把mailbox得到的张量 进行max,min,mean…… 聚合操作


# 更新函数

把聚合后的mailbox和点特征x进行一定运算，得到新的点特征x

In [4]:
# update_all(消息函数,聚合函数,更新函数=可选)
# Eg:
def update_all_example(graph):
    # 在graph.ndata['ft']中存储结果
    graph.update_all(fn.u_mul_e('ft', 'a', 'm'),
                    fn.sum('m', 'ft'))
    # 在update_all外调用更新函数
    final_ft = graph.ndata['ft'] * 2
    return final_ft

# 在Graph的一部分进行消息传递

先把子图构建出来，然后进行消息传递

# 异构图消息传递

异构图上的消息传递可以分为两个部分：

（1）对每个关系计算和聚合消息。

（2）对每个结点聚合来自不同关系的消息。

(1)multi_update_all() 接受一个字典。
 
这个字典的每一个键值对里，键是一种关系， 值是这种关系对应 update_all() 的参数。 
 
(2)multi_update_all() 还接受一个字符串来表示跨类型整合函数，来指定整合不同关系聚合结果的方式。 这个整合方式可以是 sum、 min、 max、 mean 和 stack 中的一个。

In [3]:
import dgl.function as fn

def fun():
    for c_etype in G.canonical_etypes:
        srctype, etype, dsttype = c_etype
        Wh = self.weight[etype](feat_dict[srctype])
        # 把它存在图中用来做消息传递
        G.nodes[srctype].data['Wh_%s' % etype] = Wh
        # 指定每个关系的消息传递函数：(message_func, reduce_func).
        # 注意结果保存在同一个目标特征“h”，说明聚合是逐类进行的
        funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
    # 将每个类型消息聚合的结果相加
    G.multi_update_all(funcs, 'sum')
    # 返回更新过的节点特征字典
    return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}